In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ==============================================================================
# CELL 1: TRAIN IDENTIFIER MODULE
# Uses ALL data from single_chromosomes_object folder
# ==============================================================================

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50
import xml.etree.ElementTree as ET
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict
import albumentations as A

# ==================== SHARED UTILITIES ====================

class AnnotationParser:
    """Parse XML annotations for both single and 24-chromosome datasets"""

    @staticmethod
    def parse_xml(xml_path: str) -> Dict:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        annotations = {
            'filename': root.find('filename').text if root.find('filename') is not None else '',
            'size': {},
            'objects': []
        }

        size = root.find('size')
        if size is not None:
            annotations['size'] = {
                'width': int(size.find('width').text),
                'height': int(size.find('height').text),
                'depth': int(size.find('depth').text) if size.find('depth') is not None else 3
            }

        for obj in root.findall('object'):
            name = obj.find('name').text
            bndbox = obj.find('bndbox')

            if bndbox is not None:
                bbox = {
                    'xmin': int(float(bndbox.find('xmin').text)),
                    'ymin': int(float(bndbox.find('ymin').text)),
                    'xmax': int(float(bndbox.find('xmax').text)),
                    'ymax': int(float(bndbox.find('ymax').text))
                }

                annotations['objects'].append({
                    'name': name,
                    'bbox': bbox
                })

        return annotations


# ==================== IDENTIFIER MODULE ====================

class ChromosomeIdentifierDataGenerator(keras.utils.Sequence):
    """Data generator for training chromosome identifier"""

    def __init__(self, images_dir: str, annotations_dir: str,
                 batch_size=2, img_size=800, shuffle=True):
        self.images_dir = images_dir
        self.annotations_dir = annotations_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.parser = AnnotationParser()

        # Get all image files from directory
        self.image_list = []
        for filename in os.listdir(images_dir):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                base_name = os.path.splitext(filename)[0]
                xml_path = os.path.join(annotations_dir, f"{base_name}.xml")
                if os.path.exists(xml_path):
                    self.image_list.append(base_name)

        print(f"Found {len(self.image_list)} images for identifier training")

        self.indexes = np.arange(len(self.image_list))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.ceil(len(self.image_list) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_images = [self.image_list[k] for k in batch_indexes]

        X, y = self._generate_data(batch_images)
        return X, y

    def _generate_data(self, batch_images):
        images = []
        targets = []

        for img_name in batch_images:
            img_path = os.path.join(self.images_dir, f"{img_name}.jpg")
            xml_path = os.path.join(self.annotations_dir, f"{img_name}.xml")

            if not os.path.exists(img_path):
                img_path = os.path.join(self.images_dir, f"{img_name}.png")

            if not os.path.exists(img_path) or not os.path.exists(xml_path):
                continue

            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            orig_h, orig_w = image.shape[:2]

            image = cv2.resize(image, (self.img_size, self.img_size))
            image = image.astype(np.float32) / 255.0

            annotations = self.parser.parse_xml(xml_path)

            boxes = []
            for obj in annotations['objects']:
                bbox = obj['bbox']
                xmin = bbox['xmin'] / orig_w
                ymin = bbox['ymin'] / orig_h
                xmax = bbox['xmax'] / orig_w
                ymax = bbox['ymax'] / orig_h
                boxes.append([ymin, xmin, ymax, xmax])

            images.append(image)
            targets.append(np.array(boxes) if len(boxes) > 0 else np.zeros((0, 4)))

        return images, targets

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


class ChromosomeIdentifier:
    """Chromosome detection model"""

    def __init__(self, img_size=800, max_detections=50):
        self.img_size = img_size
        self.max_detections = max_detections
        self.model = self._build_model()

    def _build_model(self):
        inputs = layers.Input(shape=(self.img_size, self.img_size, 3))

        base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)

        for layer in base_model.layers[:100]:
            layer.trainable = False

        x = base_model.output
        x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)
        x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)
        x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(0.5)(x)

        bbox_output = layers.Dense(self.max_detections * 4, name='bbox')(x)
        objectness = layers.Dense(self.max_detections, activation='sigmoid', name='objectness')(x)

        model = models.Model(inputs=inputs, outputs=[bbox_output, objectness])
        return model

    def train(self, train_gen, epochs=10):
        optimizer = optimizers.Adam(0.001)

        for epoch in range(epochs):
            print(f"\nEpoch {epoch+1}/{epochs}")
            epoch_losses = []

            for batch_idx in range(len(train_gen)):
                images, targets = train_gen[batch_idx]

                if len(images) == 0:
                    continue

                images = np.array(images)

                with tf.GradientTape() as tape:
                    bbox_pred, obj_pred = self.model(images, training=True)
                    bbox_pred = tf.reshape(bbox_pred, (-1, self.max_detections, 4))

                    batch_loss = 0
                    for i, target_boxes in enumerate(targets):
                        if len(target_boxes) > 0:
                            num_objs = min(len(target_boxes), self.max_detections)

                            target_padded = np.zeros((self.max_detections, 4))
                            target_padded[:num_objs] = target_boxes[:num_objs]

                            bbox_loss = tf.reduce_mean(tf.square(bbox_pred[i] - target_padded))

                            obj_target = np.zeros(self.max_detections)
                            obj_target[:num_objs] = 1
                            obj_loss = tf.keras.losses.binary_crossentropy(obj_target, obj_pred[i])

                            batch_loss += bbox_loss + 0.5 * tf.reduce_mean(obj_loss)

                    batch_loss = batch_loss / len(targets)

                grads = tape.gradient(batch_loss, self.model.trainable_variables)
                optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

                epoch_losses.append(float(batch_loss))

                if (batch_idx + 1) % 10 == 0:
                    print(f"  Batch {batch_idx+1}/{len(train_gen)}, Loss: {np.mean(epoch_losses[-10:]):.4f}")

            print(f"Epoch Loss: {np.mean(epoch_losses):.4f}")

        return self.model

    def predict_boxes(self, image, confidence_threshold=0.3):
        orig_h, orig_w = image.shape[:2]
        image_resized = cv2.resize(image, (self.img_size, self.img_size))
        image_resized = image_resized.astype(np.float32) / 255.0
        image_resized = np.expand_dims(image_resized, axis=0)

        bbox_pred, obj_pred = self.model.predict(image_resized, verbose=0)

        bbox_pred = bbox_pred.reshape(self.max_detections, 4)
        obj_pred = obj_pred.reshape(self.max_detections)

        keep = obj_pred > confidence_threshold
        boxes = bbox_pred[keep]
        scores = obj_pred[keep]

        final_boxes = []
        for box in boxes:
            ymin, xmin, ymax, xmax = box
            xmin = int(xmin * orig_w)
            ymin = int(ymin * orig_h)
            xmax = int(xmax * orig_w)
            ymax = int(ymax * orig_h)

            xmin = max(0, min(xmin, orig_w))
            xmax = max(0, min(xmax, orig_w))
            ymin = max(0, min(ymin, orig_h))
            ymax = max(0, min(ymax, orig_h))

            if xmax > xmin and ymax > ymin:
                final_boxes.append([xmin, ymin, xmax, ymax])

        return np.array(final_boxes), scores[:len(final_boxes)]


# ==================== TRAIN IDENTIFIER ====================

print("="*60)
print("CELL 1: TRAINING CHROMOSOME IDENTIFIER")
print("="*60)

DATA_ROOT = '/content/drive/MyDrive/ParadoX/inter_iit/2025_Karyogram_CV_Camp'
SINGLE_CHR_IMAGES = os.path.join(DATA_ROOT, 'single_chromosomes_object', 'images')
SINGLE_CHR_ANNOTATIONS = os.path.join(DATA_ROOT, 'single_chromosomes_object', 'annotations')

print("\nTensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

train_identifier_gen = ChromosomeIdentifierDataGenerator(
    SINGLE_CHR_IMAGES, SINGLE_CHR_ANNOTATIONS,
    batch_size=2, img_size=800, shuffle=True
)

identifier = ChromosomeIdentifier(img_size=800, max_detections=50)

print("\nStarting identifier training...")
identifier.train(train_identifier_gen, epochs=10)

identifier.model.save('identifier_model.h5')
print("\n✓ Identifier model saved to 'identifier_model.h5'")
print("="*60)


CELL 1: TRAINING CHROMOSOME IDENTIFIER

TensorFlow version: 2.19.0
GPU Available: []
Found 3 images for identifier training
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

Starting identifier training...

Epoch 1/10
Epoch Loss: 1.4368

Epoch 2/10
Epoch Loss: 0.5887

Epoch 3/10
Epoch Loss: 0.5151

Epoch 4/10
Epoch Loss: 0.3998

Epoch 5/10
Epoch Loss: 0.3864

Epoch 6/10
Epoch Loss: 0.3188

Epoch 7/10
Epoch Loss: 0.2629

Epoch 8/10
Epoch Loss: 0.2358

Epoch 9/10
Epoch Loss: 0.2139

Epoch 10/10


Epoch Loss: 0.2037

✓ Identifier model saved to 'identifier_model.h5'


In [14]:
# ==============================================================================
# CELL 2: TRAIN CLASSIFIER MODULE
# Uses ALL data from 24_chromosomes_object folder
# ==============================================================================

class ChromosomeClassifierDataGenerator(keras.utils.Sequence):
    """Data generator for training chromosome classifier"""

    CHROMOSOME_CLASSES = {
        '1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5,
        '7': 6, '8': 7, '9': 8, '10': 9, '11': 10, '12': 11,
        '13': 12, '14': 13, '15': 14, '16': 15, '17': 16, '18': 17,
        '19': 18, '20': 19, '21': 20, '22': 21, 'X': 22, 'Y': 23
    }

    def __init__(self, images_dir: str, annotations_dir: str, is_validation=False,
                 batch_size=32, img_size=224, augment=False, validation_split=0.2,
                 data=None, indexes=None): # Add data and indexes parameters
        print(f"\nInitializing ChromosomeClassifierDataGenerator:")
        self.images_dir = images_dir
        self.annotations_dir = annotations_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.augment = augment
        self.validation_split = validation_split
        self.is_validation = is_validation
        self.parser = AnnotationParser()

        # Initialize data storage
        if data is None or indexes is None:
            self.chromosome_data = []
            self._build_dataset()

            # Split dataset only for the training generator
            if not self.is_validation:
                total_samples = len(self.chromosome_data)
                val_size = int(total_samples * validation_split)

                # Create train/val indexes
                np.random.seed(42)
                all_indexes = np.arange(total_samples)
                np.random.shuffle(all_indexes)

                self.indexes = all_indexes[val_size:]
                self.val_indexes = all_indexes[:val_size]

                print(f"Train samples: {len(self.indexes)}, Validation samples: {len(self.val_indexes)}")
            else:
                 # For validation generator, use all data passed to it
                 self.chromosome_data = data
                 self.indexes = indexes
                 self.val_indexes = [] # Not needed for validation generator
                 print(f"Validation samples: {len(self.indexes)}")
        else:
            # Use provided data and indexes for validation generator
            self.chromosome_data = data
            self.indexes = indexes
            self.val_indexes = [] # Not needed for validation generator
            print(f"Validation samples: {len(self.indexes)}")


        # Setup augmentation if required
        if augment and not self.is_validation: # Apply augmentation only to training data
            self.aug = A.Compose([
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.Rotate(limit=15, p=0.5),
                A.RandomBrightnessContrast(p=0.3),
                A.GaussNoise(p=0.2),
            ])
        else:
            self.aug = None


    def _build_dataset(self):
        """Build dataset from images and annotations"""
        print("\nBuilding classifier dataset...")

        # Get list of valid image files
        image_files = []
        for filename in os.listdir(self.images_dir):
            if filename.endswith(('.jpg', '.png')):
                base_name = os.path.splitext(filename)[0]
                xml_path = os.path.join(self.annotations_dir, f"{base_name}.xml")
                if os.path.exists(xml_path):
                    image_files.append(base_name)

        print(f"Found {len(image_files)} images to process...")

        # Process each image and its annotations
        for img_name in image_files:
            img_path = os.path.join(self.images_dir, f"{img_name}.jpg")
            xml_path = os.path.join(self.annotations_dir, f"{img_name}.xml")

            # Try PNG if JPG doesn't exist
            if not os.path.exists(img_path):
                img_path = os.path.join(self.images_dir, f"{img_name}.png")

            if not os.path.exists(img_path) or not os.path.exists(xml_path):
                print(f"Warning: Missing files for {img_name}")
                continue

            # Load and process image
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Parse annotations
            annotations = self.parser.parse_xml(xml_path)

            # Extract individual chromosomes
            for obj in annotations['objects']:
                bbox = obj['bbox']
                label = obj['name']

                if label in self.CHROMOSOME_CLASSES:
                    chromosome_img = image[bbox['ymin']:bbox['ymax'],
                                        bbox['xmin']:bbox['xmax']]

                    if chromosome_img.size > 0:
                        self.chromosome_data.append({
                            'image': chromosome_img,
                            'label': self.CHROMOSOME_CLASSES[label]
                        })

        print(f"Built dataset with {len(self.chromosome_data)} chromosome samples")


    def get_validation_generator(self):
        """Create validation generator using validation split"""
        # Create a new generator instance specifically for validation
        val_gen = ChromosomeClassifierDataGenerator(
            images_dir=self.images_dir, # Pass directories (not strictly needed but keeps signature)
            annotations_dir=self.annotations_dir,
            batch_size=self.batch_size,
            img_size=self.img_size,
            augment=False, # No augmentation for validation
            validation_split=0.0, # Not used for validation generator
            is_validation=True, # Mark as validation generator
            data=[self.chromosome_data[i] for i in self.val_indexes], # Pass the already built data
            indexes=np.arange(len(self.val_indexes)) # Pass the validation indexes
        )
        return val_gen


    def __len__(self):
        """Return number of batches per epoch"""
        return int(np.ceil(len(self.indexes) / self.batch_size))

    def __getitem__(self, index):
        """Get batch of data"""
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        batch_images = []
        batch_labels = []

        for idx in batch_indexes:
            data = self.chromosome_data[idx]
            image = data['image'].copy()
            label = data['label']

            # Apply augmentation if enabled
            if self.aug is not None:
                augmented = self.aug(image=image)
                image = augmented['image']

            # Preprocess image
            image = cv2.resize(image, (self.img_size, self.img_size))
            image = image.astype(np.float32) / 255.0

            # Normalize using ImageNet stats
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            image = (image - mean) / std

            batch_images.append(image)
            batch_labels.append(label)

        return np.array(batch_images), np.array(batch_labels)

    def on_epoch_end(self):
        """Called at the end of every epoch"""
        if not self.is_validation: # Only shuffle training data
            np.random.shuffle(self.indexes)


class ChromosomeClassifier:
    """Chromosome classification model (24 classes)"""

    def __init__(self, num_classes=24, img_size=224):
        self.num_classes = num_classes
        self.img_size = img_size
        self.model = self._build_model()

    def _build_model(self):
        base_model = ResNet50(include_top=False, weights='imagenet',
                             input_shape=(self.img_size, self.img_size, 3))

        for layer in base_model.layers[:140]:
            layer.trainable = False

        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(0.3)(x)
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)

        model = models.Model(inputs=base_model.input, outputs=outputs)
        return model

    def compile_model(self, lr=0.001):
        self.model.compile(
            optimizer=optimizers.Adam(lr),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

    def train(self, train_gen, val_gen, epochs=30):
        callbacks = [
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_accuracy', factor=0.5, patience=3,
                min_lr=1e-7, verbose=1
            ),
            keras.callbacks.ModelCheckpoint(
                'best_classifier.h5', monitor='val_accuracy',
                save_best_only=True, verbose=1
            ),
            keras.callbacks.EarlyStopping(
                monitor='val_accuracy', patience=7,
                restore_best_weights=True, verbose=1
            )
        ]

        history = self.model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1
        )

        return history


# ==================== TRAIN CLASSIFIER ====================

print("\n" + "="*60)
print("CELL 2: TRAINING CHROMOSOME CLASSIFIER")
print("="*60)

DATA_ROOT = '/content/drive/MyDrive/ParadoX/inter_iit/2025_Karyogram_CV_Camp'
MULTI_CHR_IMAGES = os.path.join(DATA_ROOT, '24_chromosomes_object', 'images')
MULTI_CHR_ANNOTATIONS = os.path.join(DATA_ROOT, '24_chromosomes_object', 'annotations')

train_classifier_gen = ChromosomeClassifierDataGenerator(
    MULTI_CHR_IMAGES, MULTI_CHR_ANNOTATIONS,
    batch_size=32, img_size=224, augment=True, validation_split=0.2
)

val_classifier_gen = train_classifier_gen.get_validation_generator()

classifier = ChromosomeClassifier(num_classes=24)
classifier.compile_model(lr=0.001)

print("\nStarting classifier training...")
history = classifier.train(train_classifier_gen, val_classifier_gen, epochs=30)

print("\n✓ Classifier model saved to 'best_classifier.h5'")
print("="*60)


CELL 2: TRAINING CHROMOSOME CLASSIFIER

Initializing ChromosomeClassifierDataGenerator:

Building classifier dataset...
Found 10 images to process...
Built dataset with 18 chromosome samples
Train samples: 15, Validation samples: 3

Initializing ChromosomeClassifierDataGenerator:
Validation samples: 3

Starting classifier training...
Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39s/step - accuracy: 0.0667 - loss: 3.2035
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to best_classifier.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 44s 44s/step - accuracy: 0.0667 - loss: 3.2035 - val_accuracy: 1.0000 - val_loss: 0.7895 - learning_rate: 0.0010
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.7333 - loss: 1.3299
Epoch 2: val_accuracy did not improve from 1.00000
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step - accuracy: 0.7333 - loss: 1.3299 - val_accuracy: 1.0000 - val_loss: 0.0380 - learning_rate: 0.0010
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.8000 - loss: 0.5234
Epoch 3: val_accuracy did not improve from 1.00000
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.8000 - loss: 0.5234 - val_accuracy: 1.0000 - val_loss: 9.8262e-05 - learning_rate: 0.0010
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.8667 - loss: 0.1926
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 4: val_accuracy did not improve from 1.00000
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.8667 - loss: 0.1926 - val_accuracy: 1.0000 - val_loss: 5.086

In [15]:
# ==============================================================================
# CELL 3: INTEGRATION AND EVALUATION
# Uses train.txt and test.txt to select specific images from 24_chromosomes_object
# ==============================================================================

class KaryotypePipeline:
    """End-to-end pipeline integrating identifier and classifier"""

    def __init__(self, identifier: ChromosomeIdentifier,
                 classifier: ChromosomeClassifier):
        self.identifier = identifier
        self.classifier = classifier
        self.img_size = 224

    def process_image(self, image_path: str, confidence_threshold=0.3):
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        boxes, scores = self.identifier.predict_boxes(image_rgb, confidence_threshold=confidence_threshold)

        if len(boxes) == 0:
            return []

        predictions = []

        for i, box in enumerate(boxes):
            xmin, ymin, xmax, ymax = map(int, box)

            chromosome_img = image_rgb[ymin:ymax, xmin:xmax]

            if chromosome_img.size == 0:
                continue

            chromosome_img = cv2.resize(chromosome_img, (self.img_size, self.img_size))
            chromosome_img = chromosome_img.astype(np.float32) / 255.0

            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            chromosome_img = (chromosome_img - mean) / std

            chromosome_img = np.expand_dims(chromosome_img, axis=0)

            probs = self.classifier.model.predict(chromosome_img, verbose=0)[0]
            class_id = np.argmax(probs)
            confidence = probs[class_id]

            predictions.append({
                'bbox': box,
                'class_id': int(class_id),
                'class_name': self._get_class_name(class_id),
                'confidence': float(confidence),
                'score': float(scores[i]) if i < len(scores) else 1.0
            })

        return predictions

    @staticmethod
    def _get_class_name(class_id: int) -> str:
        if class_id < 22:
            return str(class_id + 1)
        elif class_id == 22:
            return 'X'
        else:
            return 'Y'


def evaluate_pipeline(pipeline: KaryotypePipeline, images_dir: str,
                     annotations_dir: str, test_list: List[str],
                     output_dir='results'):
    os.makedirs(output_dir, exist_ok=True)

    parser = AnnotationParser()

    all_true_labels = []
    all_pred_probs = []

    print("Evaluating pipeline on test set...")
    for idx, img_name in enumerate(test_list):
        img_name = img_name.strip()
        img_path = os.path.join(images_dir, f"{img_name}.jpg")
        xml_path = os.path.join(annotations_dir, f"{img_name}.xml")

        if not os.path.exists(img_path):
            img_path = os.path.join(images_dir, f"{img_name}.png")

        if not os.path.exists(img_path) or not os.path.exists(xml_path):
            continue

        if (idx + 1) % 10 == 0:
            print(f"Processed {idx+1}/{len(test_list)} images...")

        annotations = parser.parse_xml(xml_path)
        gt_chromosomes = [obj['name'] for obj in annotations['objects']]

        predictions = pipeline.process_image(img_path)

        for gt_chr in gt_chromosomes:
            if gt_chr in ChromosomeClassifierDataGenerator.CHROMOSOME_CLASSES:
                gt_class = ChromosomeClassifierDataGenerator.CHROMOSOME_CLASSES[gt_chr]

                true_labels = np.zeros(24)
                true_labels[gt_class] = 1

                pred_probs = np.zeros(24)
                for pred in predictions:
                    pred_probs[pred['class_id']] = max(
                        pred_probs[pred['class_id']],
                        pred['confidence']
                    )

                all_true_labels.append(true_labels)
                all_pred_probs.append(pred_probs)

    all_true_labels = np.array(all_true_labels)
    all_pred_probs = np.array(all_pred_probs)

    auprc_scores = []
    plt.figure(figsize=(15, 10))

    for i in range(24):
        if np.sum(all_true_labels[:, i]) > 0:
            precision, recall, _ = precision_recall_curve(
                all_true_labels[:, i],
                all_pred_probs[:, i]
            )
            auprc = auc(recall, precision)
            auprc_scores.append(auprc)

            plt.plot(recall, precision,
                    label=f'Class {pipeline._get_class_name(i)} (auPRC={auprc:.3f})')

    plt.xlabel('Recall', fontsize=12)
    plt.ylabel('Precision', fontsize=12)
    plt.title('Precision-Recall Curves for All Chromosome Classes', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'auprc_curves.png'),
                dpi=300, bbox_inches='tight')
    plt.close()

    mean_auprc = np.mean(auprc_scores)
    print(f"\nMean auPRC: {mean_auprc:.4f}")

    with open(os.path.join(output_dir, 'auprc_scores.txt'), 'w') as f:
        f.write(f"Mean auPRC: {mean_auprc:.4f}\n\n")
        f.write("Per-class auPRC scores:\n")
        for i, score in enumerate(auprc_scores):
            f.write(f"Class {pipeline._get_class_name(i)}: {score:.4f}\n")

    return mean_auprc, auprc_scores


# ==================== INTEGRATION AND EVALUATION ====================

print("\n" + "="*60)
print("CELL 3: INTEGRATION AND EVALUATION")
print("="*60)

# Load train/test splits from txt files
print("\nLoading train/test splits...")
with open(os.path.join(DATA_ROOT, 'train.txt'), 'r') as f:
    train_list = f.readlines()

with open(os.path.join(DATA_ROOT, 'test.txt'), 'r') as f:
    test_list = f.readlines()

print(f"Train samples from train.txt: {len(train_list)}")
print(f"Test samples from test.txt: {len(test_list)}")

# Load trained models
print("\nLoading trained models...")
identifier_loaded = ChromosomeIdentifier(img_size=800, max_detections=50)
identifier_loaded.model = keras.models.load_model('identifier_model.h5')
print("✓ Identifier model loaded")

classifier_loaded = ChromosomeClassifier(num_classes=24)
classifier_loaded.model = keras.models.load_model('best_classifier.h5')
print("✓ Classifier model loaded")

# Create pipeline
pipeline = KaryotypePipeline(identifier_loaded, classifier_loaded)
print("\n✓ Pipeline created successfully")

# Evaluate on test set (using images specified in test.txt)
print("\nEvaluating pipeline on test set from test.txt...")
mean_auprc, class_auprcs = evaluate_pipeline(
    pipeline, MULTI_CHR_IMAGES, MULTI_CHR_ANNOTATIONS,
    test_list, output_dir='results'
)

print("\n" + "="*60)
print("EVALUATION COMPLETE!")
print("="*60)
print(f"Mean auPRC: {mean_auprc:.4f}")
print(f"\nResults saved to 'results/' directory:")
print(f"  - auPRC curve: results/auprc_curves.png")
print(f"  - Detailed scores: results/auprc_scores.txt")
print("="*60)


CELL 3: INTEGRATION AND EVALUATION

Loading train/test splits...


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ParadoX/inter_iit/2025_Karyogram_CV_Camp/train.txt'